# Machine Learning Project

__Pia CHANCEREL - Raphael LASRY - Maxime POLI__

Based on the article :

_A Continuation Method for Semi-Supervised SVMs_

Olivier Chapelle $\hspace{3.9cm}$ olivier.chapelle@tuebingen.mpg.de

Mingmin Chi $\hspace{4.5cm}$ mingmin.chi@tuebingen.mpg.de

Alexander Zien $\hspace{4.1cm}$ alexander.zien@tuebingen.mpg.de


Max Planck Institute for Biological Cybernetics, Tübingen, Germany

https://dl.acm.org/doi/pdf/10.1145/1143844.1143868?download=true

In [1]:
import numpy as np
import scipy as sc

# Data

20newsgroup dataset: https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html

Dataset of an old forum. We will only focus on messages related to windows and mac. The goal is to predict the subject of the message (windows or mac) thanks to a $S^3VM$ method.

In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import PolynomialFeatures

In [3]:
cat = ['comp.sys.mac.hardware', 'comp.os.ms-windows.misc']
newsgroups_train = fetch_20newsgroups(subset = 'train', categories = cat) 
newsgroups_test = fetch_20newsgroups(subset = 'test', categories = cat) 

In [4]:
# print(newsgroups_train.DESCR) # Documentation of how to use the dataset

In [5]:
print('training set')
print('posts', newsgroups_train.filenames.shape) # Text, content of the messages
print('class', newsgroups_train.target.shape) # In which categories the message should be classified
print('test set')
print('posts', newsgroups_test.filenames.shape)
print('class', newsgroups_test.target.shape)

training set
posts (1169,)
class (1169,)
test set
posts (779,)
class (779,)


In [6]:
# Converting text to vectors
vectorizer = TfidfVectorizer()
vectors_train = vectorizer.fit_transform(newsgroups_train.data) # From a text message to a sparse matrix
vectors_test = vectorizer.fit_transform(newsgroups_test.data)
print(vectors_train.shape)
print(vectors_test.shape)

(1169, 48026)
(779, 26537)


In [7]:
print(vectors_train) # Sparse matrix. The first index is the index of the message and the second ones are the indexes whithin this matrix where the value isn't a 0.

  (0, 11836)	0.060745952137355695
  (0, 24291)	0.022618106900418693
  (0, 41388)	0.11860995066421275
  (0, 31786)	0.03240293556665717
  (0, 29765)	0.035913087826464844
  (0, 39120)	0.07022124808552059
  (0, 16510)	0.05070982504718354
  (0, 35013)	0.048867754439260905
  (0, 24807)	0.04686961078206869
  (0, 20384)	0.05719755934000299
  (0, 41675)	0.039978081454834856
  (0, 11678)	0.05598156693760046
  (0, 22670)	0.027010341186943107
  (0, 26672)	0.03835102912668531
  (0, 46908)	0.08984813071411515
  (0, 24015)	0.030345755269912806
  (0, 9834)	0.1237192022846886
  (0, 15131)	0.1309202879936192
  (0, 21441)	0.1237192022846886
  (0, 27940)	0.09767696412557507
  (0, 16527)	0.11860995066421275
  (0, 16440)	0.1309202879936192
  (0, 955)	0.07485176283271351
  (0, 24739)	0.04527614649220864
  (0, 41399)	0.05820447219377972
  :	:
  (1168, 35408)	0.06641869967872376
  (1168, 37311)	0.08705716680767477
  (1168, 20022)	0.059099265254513346
  (1168, 45316)	0.07897469031592323
  (1168, 1041)	0.0650417

# S3VM

In [8]:
X = np.ones((3, 4))
Y = np.arange(3)
print(np.multiply(X.T, Y).T)
(X.T * Y).T

[[0. 0. 0. 0.]
 [1. 1. 1. 1.]
 [2. 2. 2. 2.]]


array([[0., 0., 0., 0.],
       [1., 1., 1., 1.],
       [2., 2., 2., 2.]])

In [33]:
class S3VMClassifier():
    """
        S3VM class as defined in the article.
    """
    def __init__(self, C=1.0, C_=1.0, lr=0.01, s=3):
        self.C = C
        self.C_ = C_
        self.lr = lr
        self.s = s
        self.X = None
        self.Y = None
        self.X_ = None
        self.w = None
        self.b = None
        
    def compute_gamma(self, epochs):
        """
            Compute the list of values of gamma (defined at the §3.2).
        """
        m, d = self.X_.shape
        matrix = np.zeros((d, d))
        for i in range(m):
            matrix += np.outer(self.X_[i], self.X_[i]) / np.linalg.norm(self.X_[i]) ** 3
        lamb_max = numpy.linalg.eigh(matrix)[0][- 1]
        gamma_0 = (self.C_ * lamb_max) ** (2 / 3) / (2 * self.s) ** (1 / 3)
        gamma_end = 1 / (epochs * 2 * self.s * sc.sparse.linalg.norm(self.X_, axis=1).max() ** 2)
        gamma_list = [(gamma_end / gamma_0) ** (i / epochs) * gamma_0 for i in range(epochs)]
        return gamma_list
    
    def loss(self, gamma):
        """
            Compute the convolved loss (defined at the end of the §3.1).
        """
        d = self.X.shape[1]
        a = 1 + 2 * gamma * self.s * np.linalg.norm(self.X_, axis = 1) ** 2
        e = (Y * (self.X.dot(self.w) + self.b) - 1) / np.sqrt(2*gamma) / np.linalg.norm(self.X, axis = 1)
        L_labelled = self.C * np.sum(gamma * np.linalg.norm(self.X, axis = 1) / np.sqrt(2) 
                                     * (np.exp(- e**2) / np.sqrt(np.pi) - e * sc.special.erfc(e)))
        L_unlabelled = self.C_ * np.sum(1 / np.sqrt(a) * np.exp(- self.s * (self.X_.dot(self.w) + self.b) ** 2 / a))
        return np.dot(self.w, self.w)/2 + gamma*d/2 + L_labelled + L_unlabelled
    
    def gradient_loss(self, gamma, labelled):
        """
            Compute the gradient of the loss (defined at the end of the §3.1).
        """
        a = 1 + 2 * gamma * self.s * np.linalg.norm(self.X_, axis = 1) ** 2
        e = (Y * (self.X.dot(self.w) + self.b) - 1) / np.sqrt(2 * gamma) / np.linalg.norm(self.X, axis = 1)
        if labelled:
            dL_labelled = self.C / 2 * np.sum(np.multiply(X.T, sc.special.erfc(e) * self.Y).T, axis=0)
            return self.w - dL_labelled
        else:
            dL_unlabelled = self.C_ * np.sum(np.multiply(X.T, 2 * self.s * (self.X_.dot(self.w) + self.b) / a ** (3 / 2)
                                         * np.exp(- self.s * (self.X_.dot(self.w) + self.b) ** 2 / a)), axis=0)
            return self.w - dL_unlabelled
    
    def minimize(self, gamma, epochs):
        """
            Optimise the parameters.
        """
        n, d, m = self.X.shape, self.X_.shape[0]
        indexes = np.arange(n + m) - m # positive indexes --> labeled & negative indexes --> unlabeled
        gw = np.zeros((n + m, d)) # vector of most recent gradient in w
        dw = np.zeros(d) # sum of gw
        for ep in range(epochs):
            np.random.shuffle(indexes)
            for k in range(n + m):
                i = indexes[k]
                labelled = (i >= 0) # labelled or unlabelled
                if i < 0 : # if it's the index of an unlabelled data
                    i = - i + n # its index is in gw
                dw -= gw[i]
                gw[i] = self.gradient_loss(gamma, labelled)
                dw += gw[i]
                self.w -= self.lr / min((k + 1) + ep * (n + m), n + m) * dw
    
    def fit(self, X, Y, X_, epochs=5, iter_gamma=10): # epochs pour la minimisation, iter_gamma pour le nombre de gamma
        """
            Train the model on X_labelled, Y_labelled, X_unlabelled datas.
        """
        self.X = X
        self.Y = Y
        self.X_ = X_ - np.mean(X_)
        
        gamma_list = self.compute_gamma(iter_gamma)
        # warm start
        if (w is None) and (b is None) :
            _, d = self.X.shape
            self.w = np.zeros(d)
            self.b = np.mean(Y)
        # continuation method
        for gamma in gamma_list :
            self.minimize(gamma, epochs)

    def predict(self, X):
        """
            Predict the value of Y (0 or 1).
        """
        return self.w.T.dot(X) + self.b >= 0
    
    def accuracy(self, X, Y):
        """
            Evaluate the accuracy of the prediction.
        """
        return (self.predict(X) == Y).mean()

In [34]:
s3vm = S3VMClassifier()
s3vm.fit(vectors_train[:int(0.75 * vectors_train.shape[0]), :], newsgroups_train.target[:int(0.75 * vectors_train.shape[0])], vectors_train[int(0.75 * vectors_train.shape[0]):, :])
print(s3vm.accuracy(vectors_test, newsgroups_test))

NotImplementedError: subtracting a nonzero scalar from a sparse matrix is not supported